In [5]:
import time
import json
import requests as r
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from Queue import Queue
from threading import Thread


hotels_com_list=[]
with open('hotels3.json','r') as f:
    hotels_url=json.load(f)
area_queue = Queue()
for area in hotels_url:
    area_queue.put(area['url'])
def worker():
    while not area_queue.empty():
        crawler() 

def crawler():
    for myd_url in hotels_url:
        hotels_com={}
        URL= area_queue.get()
        driver = webdriver.Remote("http://localhost:9515", webdriver.DesiredCapabilities.CHROME)
        
        try:
            driver.get(URL)
            time.sleep(1)
            driver.maximize_window()
            soup=BeautifulSoup(driver.page_source,'lxml')
            URL1='https://tw.hotels.com'+soup.select('.total-reviews')[0]['href']
            hotels_com['hotel']=soup.select('h1')[0].text
            hotels_com['price']=soup.select('.featured-price')[0].text
            #print hotels_com['price']            
            hotels_com['address']=soup.select('.postal-addr')[0].text
            res=r.get(URL1)
            soup2=BeautifulSoup(res.text , 'lxml')
            hotels_com['level']=soup2.select('.rating')[0].text

            l=len(soup2.select('.review-card-meta-reviewer'))-1
            hotels_com['comment']=[]
            for i in range(0 ,l): 
                hotels={'name':soup2.select('.review-card-meta-reviewer')[i].text, 'comm':soup2.select('.expandable-content')[i].text}   
                hotels_com['comment'].append(hotels)
            hotels_com_list.append(hotels_com)
            #print hotels_com_list
            print hotels_com['level']
            if area_queue.empty():
                driver.close()
            else:
                driver.close()
                print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
                worker()
        except:
            driver.close()
            worker()
        
               

threads = map(lambda i: Thread(target=worker), xrange(4))
map(lambda th: th.start(), threads)
map(lambda th: th.join(), threads)
hotels1 = json.dumps(hotels_com_list, ensure_ascii=False) # This is a unicode string
with open('hotels_com1.json', 'w') as f1:
    f1.write(hotels1.encode('utf-8'))



In [ ]:
with open('hotels_com1.json','r') as f:
    hotels5=json.load(f)
len(hotels5)

In [ ]:
import json
hotels1 = json.dumps(hotels_com_list, ensure_ascii=False) # This is a unicode string
with open('hotels_com4.json', 'w') as f:
    f.write(hotels1.encode('utf-8'))

In [ ]:
with open('hotels_com4.json','r') as f:
    hotels5=json.load(f)
len(hotels5)